# Entrenando una red neuronal en MNIST con Keras

Este sencillo ejemplo demuestra cómo conectar conjuntos de datos de TensorFlow (TFDS) a un modelo de Keras.


Copyright 2020 Los autores de los conjuntos de datos de TensorFlow, con licencia Apache, versión 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/JhennerTigreros/UNICIENCIA-INTRO-PROGRAMACION/blob/master/tensorflow/clase_2/tensorflow_datasets.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  </td>
</table>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds


## Paso 1: cree su canal de entrada

Comience por crear un canal de entrada eficiente utilizando los consejos de:
* The [Performance tips](https://www.tensorflow.org/datasets/performances) guide
* The [Better performance with the `tf.data` API](https://www.tensorflow.org/guide/data_performance#optimize_performance) guide


### Cargar un conjunto de datos
Cargue el conjunto de datos MNIST con los siguientes argumentos:

* `shuffle_files=True`: Los datos de MNIST solo se almacenan en un solo archivo, pero para conjuntos de datos más grandes con múltiples archivos en disco, es una buena práctica mezclarlos durante el entrenamiento.
* `as_supervised=True`: Devuelve una tupla `(img, label)` en lugar de un diccionario `{'image': img, 'label': label}`.


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

### Construir una tubería de entrenamiento
 
Aplique las siguientes transformaciones:

* `tf.data.Dataset.map`: TFDS proporciona imágenes de tipo `tf.uint8`, mientras que el modelo espera `tf.float32`. Por lo tanto, necesita normalizar las imágenes.
* `tf.data.Dataset.cache`: Al ajustar el conjunto de datos en memoria, cacheelo antes de mezclarlo para un mejor rendimiento.<br/>
__Nota:__ Las transformaciones aleatorias deben aplicarse después de almacenar en caché.
* `tf.data.Dataset.shuffle`: Para una verdadera aleatoriedad, establezca el búfer de mezcla al tamaño completo del conjunto de datos.<br/>
__Nota:__ Para conjuntos de datos grandes que no caben en memoria, use `buffer_size=1000` si su sistema lo permite.
* `tf.data.Dataset.batch`: Agrupe los elementos del conjunto de datos después de mezclarlos para obtener lotes únicos en cada época.
* `tf.data.Dataset.prefetch`: Es una buena práctica terminar la tubería prefetching [para el rendimiento](https://www.tensorflow.org/guide/data_performance#prefetching).


In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

### Construir una tubería de evaluación

Tu tubería de pruebas es similar a la tubería de entrenamiento con pequeñas diferencias:

 * No necesitas llamar a `tf.data.Dataset.shuffle`.
 * El almacenamiento en caché se realiza después de la agrupación en lotes porque los lotes pueden ser los mismos entre épocas.


In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Paso 2: Crear y entrenar el modelo

Conecte el pipeline de entrada de TFDS a un modelo simple de Keras, compile el modelo y entrénelo.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

In [ ]:
# Let's first import matplotlib to help us plot the images
import matplotlib.pyplot as plt
import numpy as np

# Fetch one batch of the test data
for test_images, test_labels in ds_test.take(1):
    # Take one random image from the batch
    random_index = np.random.randint(0, test_images.shape[0])
    test_image = test_images[random_index].numpy().squeeze()
    test_label = test_labels[random_index].numpy()

# Plot the test image
plt.figure()
plt.imshow(test_image, cmap=plt.cm.binary)
plt.title('Actual Label: ' + str(test_label))
plt.show()

# Predict the label of the test image
prediction = model.predict(tf.expand_dims(test_image, 0))

# Print the predicted and actual label
print('Predicted label:', np.argmax(prediction))
print('Actual label:', test_label)